# demo for partially observed data

- in this example, we apply S3ID **spatiotemporally subsampled data** in the form of three sequentially observed subpopulations
- we generate data from a linear dynamical system with $p=100$ observed and $n=4$ latent variables:
  $$ x_{t} = A x_{t-1} + \eta_t \\    
     y_t = C x_t + \epsilon_t, $$
     with emission noise $\epsilon_t \sim \mathcal{N}(0, R)$ for diagonal matrix $R \in \mathbb{R}_{+}^{p\times{}p}$ and innovation noise $\eta_t \sim \mathcal{N}(0, Q)$ with $Q\in\mathbb{R}^{n\times{}n}$
- we run S3ID to estimate the latent space $C\in\mathbb{R}^{p\times{}n}$, emission noise levels $R$ and time-lagged latent covariances $X_\tau = \mbox{cov}[x_{t+\tau},x_t] \in \mathbb{R}^{n\times{}n}$ 


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from S3ID import main, gen_data, ObservationScheme, principal_angle, comp_model_covariances

np.random.seed(42)


# define problem size
p, n, T = 100, 4, 1000   # number of observed variable, number of hidden variables, length of time-trace

# generate toy LDS parameters and data
pars_true, x, y, _, _ = gen_data(p,n,[],T)
y -= y.mean(axis=0) # ensure zero-mean input

plt.figure(figsize=(20,5))
plt.plot(x)
plt.xlabel('t')
plt.ylabel('x_t')
plt.title('latent dynamics (n = ' + str(n) + ')')
plt.show()

# compute first 2*n time-lagged empirical covariances
Qe = [np.cov(y[:T-k,:].T, y[k:,:].T)[p:,:p] for k in range(2*n)]

print('dynamics eigenvalues', np.linalg.eigvals(pars_true['A']))


# define observation scheme 

- here we define three overlapping subpopulations imaged in repeated sequence, each for about 1/3 of the total time

In [ ]:
# define 3 subpopulations j =1,2,3 with overlap: 
sub_pops = (np.arange(40),     # variables i =  1, ...,  40
            np.arange(30, 70), # variables i = 31, ...,  70
            np.arange(60,  p)) # variables i = 61, ..., 100

# define observation switch-times between subpopulations (starting from t=0)
obs_time = np.arange(10, T+1, 10) 

# define which populations j = 1,2,3 are obseved up to each switch-time:
obs_pops = [ i%3 for i in range(len(obs_time))]

obs_scheme = ObservationScheme(p,T,sub_pops, obs_pops,obs_time)

obs_scheme.gen_mask_from_scheme()
obs_scheme.use_mask = False

# mask data
y[np.where(1-obs_scheme.mask)] = np.nan

plt.figure(figsize=(16,5))
plt.imshow(y.T)
plt.title('masked data')
plt.ylabel('#variable')
plt.xlabel('time t')
plt.show()

# monitor subspace distance relatve to ground-truth parameters
- we track the **principal angles** between the column-spaces of ground-truth matrix $C$ and our current estimate of $C$ over epochs

In [ ]:
epochs = 150
proj_errors = np.zeros((epochs,n))
def pars_track(pars,t): 
    proj_errors[t] = principal_angle(pars_true['C'], pars[0])

# fit the model

In [ ]:
lag_range = np.arange(2*n) # matching time-lagged covariances for time-lags tau = 0, 1,.., 2*n-1  

pars_est, pars_init, traces, Qs, Om, W, t_desc = main(lag_range, n, y, 
                                                      obs_scheme, sso=True, 
                                                      parametrization='nl', # non-linear parametrization
                                                      batch_size=1, max_iter=epochs,
                                                      pars_track=pars_track, verbose=True)
print('total time was ', t_desc)

# visualize results

- for the chosen observation scheme, not all pair-wise covariances are actually observed within the data
- we can query the model fit for all of the pairwise covariances, including the non-observed ones, through
  $$ \mbox{cov}(y_t, y_t) = C X_{0} C^\top + R \\
     \mbox{cov}(y_{t+\tau}, y_t) = C X_{\tau} C^\top $$
  from the estimated matrices $C$, $X_\tau$ and $R$

In [ ]:
Qh = comp_model_covariances(pars_est, lag_range=lag_range) # model-predicted time-lagged covariances

plt.figure(figsize=(16,9))
plt.subplot(2,4,1)
plt.plot(traces[0])
plt.title('loss')
plt.xlabel('epochs')
plt.subplot(2,4,5)
plt.plot(proj_errors)
plt.title('principcal angles')
plt.xlabel('epochs')

plt.subplot(2,4,2)
plt.imshow(Qs[0], interpolation='None')    # 'Qs' contains covariances computed from the masked data
plt.title('empirical instant. cov.')
plt.subplot(2,4,3)
plt.imshow(Qh[0], interpolation='None')
plt.title('est. instant. cov.')
plt.subplot(2,4,4)
plt.plot(Qe[0][Om[0]], Qh[0][Om[0]], 'g+') # 'Qe' was computed from the unmasked data
plt.plot(Qe[0][np.invert(Om[0])], Qh[0][np.invert(Om[0])], 'r+')
plt.xlabel('emp. inst. cov.')
plt.ylabel('est. inst. cov.')
plt.legend(['observed', 'non-observ.'], loc=2)
plt.axis('square')

tau = 2*n-1
plt.subplot(2,4,6)
plt.imshow(Qs[tau], interpolation='None')
plt.title('empirical cov. at time-lag tau='+str(2*n-1))
plt.subplot(2,4,7)
plt.imshow(Qh[tau], interpolation='None')
plt.title('est. cov. at time-lag tau='+str(2*n-1))
plt.subplot(2,4,8)
plt.plot(Qe[tau][Om[tau]], Qh[tau][Om[tau]], 'g+')
plt.plot(Qe[tau][np.invert(Om[tau])], Qh[tau][np.invert(Om[tau])], 'r+')
plt.axis('square')
plt.xlabel('emp. lagged cov.')
plt.ylabel('est. lagged cov.')
plt.legend(['observed', 'non-observ.'], loc=2)
    
plt.show()